In [1]:
from google.colab import files

uploaded = files.upload()

Saving Sentiment-Analysis-Caikit-and-HuggingFace-master.zip to Sentiment-Analysis-Caikit-and-HuggingFace-master (1).zip


In [2]:
!unzip /content/Sentiment-Analysis-Caikit-and-HuggingFace-master.zip -d /content/

Archive:  /content/Sentiment-Analysis-Caikit-and-HuggingFace-master.zip
32ec5dd6e04aae219eefb9d4d0f89e4da67a7bef
replace /content/Sentiment-Analysis-Caikit-and-HuggingFace-master/README.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/Sentiment-Analysis-Caikit-and-HuggingFace-master/README.md  
  inflating: /content/Sentiment-Analysis-Caikit-and-HuggingFace-master/client.py  
  inflating: /content/Sentiment-Analysis-Caikit-and-HuggingFace-master/models/text_sentiment/config.yml  
 extracting: /content/Sentiment-Analysis-Caikit-and-HuggingFace-master/requirements.txt  
  inflating: /content/Sentiment-Analysis-Caikit-and-HuggingFace-master/start_runtime.py  
  inflating: /content/Sentiment-Analysis-Caikit-and-HuggingFace-master/text_sentiment/__init__.py  
  inflating: /content/Sentiment-Analysis-Caikit-and-HuggingFace-master/text_sentiment/__pycache__/__init__.cpython-311.pyc  
  inflating: /content/Sentiment-Analysis-Caikit-and-HuggingFace-master/text_sentiment/config.

In [3]:
!pip install -r /content/Sentiment-Analysis-Caikit-and-HuggingFace-master/requirements.txt

In [15]:
import os
import subprocess
import time

# Pindah ke direktori proyek
project_dir = "/content/Sentiment-Analysis-Caikit-and-HuggingFace-master"
os.chdir(project_dir)

# Menjalankan server gRPC sebagai subprocess dan mencetak outputnya
server_process = subprocess.Popen(
    ['python', 'start_runtime.py'], stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

# Tampilkan log server untuk memastikan bahwa server berjalan
# Read server logs continuously and print them
while True:
    output = server_process.stdout.readline()
    error = server_process.stderr.readline()

    if output:
        print(f"Server Output: {output.decode().strip()}")
    if error:
        print(f"Server Error: {error.decode().strip()}")

    # Break the loop if the server process has terminated
    if server_process.poll() is not None:
        break

    time.sleep(1)

# Memberikan waktu agar server gRPC benar-benar siap
time.sleep(15)  # Increased wait time

# Kode client untuk mengirimkan request
import grpc
from caikit.runtime.service_factory import ServicePackageFactory
from text_sentiment.data_model import TextInput

# Inisialisasi service package
inference_service = ServicePackageFactory().get_service_package(
    ServicePackageFactory.ServiceType.INFERENCE,
)
port = 8085

# Setup client
channel = grpc.insecure_channel(f"localhost:{port}")
client_stub = inference_service.stub_class(channel)

# Input dan permintaan inference
for text in ["Aku mencintai dia!", "Aku ingin makan mie goreng!"]:
    input_text_proto = TextInput(text=text).to_proto()
    request = inference_service.messages.HuggingFaceSentimentTaskRequest(
        text_input=input_text_proto
    )
    response = client_stub.HuggingFaceSentimentTaskPredict(
        request, metadata=[("mm-model-id", "text_sentiment")]
    )
    print("Text:", text)
    print("RESPONSE:", response)

# Menghentikan proses server
server_process.terminate()

Server Error: 2024-11-13 09:23:29.244260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
Server Error: 2024-11-13 09:23:29.263201: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered


_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8085: Failed to connect to remote host: connect: Connection refused (111)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8085: Failed to connect to remote host: connect: Connection refused (111)", grpc_status:14, created_time:"2024-11-13T09:23:49.643777807+00:00"}"
>